<img src="header.png" align="left"/>

# Anwendungsbeispiel Analysis and quality control of data 

Die Ziel dieses Beispieles ist es die Methoden der Datenanalyse nochmals zu zeigen und deren Notwendigkeit zu begründen.


- Datentypen und Form der Daten
- Visualisierung
- Fehlende Daten
- Statistische Werte
- Outliers und Anomalien in den Daten
- Korrelationen und Beziehungen zwischen den Features
- Untersuchung der Trainingsdaten
- Stabilere Prüfung der Modellqualität


Code und Informationen entnommen von:

- [https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba)
- [https://towardsdatascience.com/5-ways-to-detect-outliers-that-every-data-scientist-should-know-python-code-70a54335a623](https://towardsdatascience.com/5-ways-to-detect-outliers-that-every-data-scientist-should-know-python-code-70a54335a623)
- [https://github.com/Viveckh/HiPlotTutorial/blob/master/Hiplot-Tutorial.ipynb](https://github.com/Viveckh/HiPlotTutorial/blob/master/Hiplot-Tutorial.ipynb)

# Import der Module

In [ ]:
#
# Import der Module
#
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import hiplot as hip

from scipy import stats
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.cluster import DBSCAN


In [ ]:
#
# Abdrehen von Fehlermeldungen
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)

#
# Einstellen der Grösse von Diagrammen
#
plt.rcParams['figure.figsize'] = [16, 9]

# Datentypen und Form der Daten

https://numpy.org/devdocs/user/basics.types.html

In [ ]:
# 
# Laden der Daten
# 
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
iris = pd.read_csv('data/iris/iris_mutilated.csv', names=names)

In [ ]:
#
# Anzeige der Form der Daten
#
print(iris.shape)

In [ ]:
#
# Anzeige der Datentypen
#
print(iris.info())

# Visualisierung

In [ ]:
#
# Anzeige von Samples zur Visualisierung der Inhalte
#
print(iris.head())

In [ ]:
print(iris.tail())

# Fehlende Daten

In [ ]:
#
# Liste alle Reihen mit fehlenden Werten
# any(axis=1) liefert ein True, wenn eines der Features über die axis 1 True ist
# 
iris[iris.isna().any(axis=1)]

In [ ]:
#
# Ersetzen durch Mittelwert des Features (grauslich)
#
iris_non = iris.fillna(iris.mean())

In [ ]:
iris_non[iris_non.isna().any(axis=1)]

Füllen von fehlenden Werte mit dem Mittelwert des Features erzeugt neue Datenpunkte, die potentiell störend sind. Alternativ können die Datenpunkte gelöscht werden.

# Duplikate

In [ ]:
#
# Prüfung auf Duplikate zeigt zwar doppelte Werte, aber keine massiven Störungen
# bis auf Weiteres keine Änderung 
#
iris_non[iris_non.duplicated()]

# Einfache Statistiken

In [ ]:
#
# Verteilung der Labels (Klassen)
#
print(iris_non.groupby('class').size())

In [ ]:
#
# Histogramm der Klassen als Plot 
#
#
# Prüfen der Verteilung der Klassen
#
df = pd.DataFrame(iris_non,columns=['class'])
counts= df.groupby('class').size()
class_pos = np.arange(3)
plt.bar(class_pos, counts, align='center', alpha=0.4)
plt.xlabel(class_pos)
plt.ylabel('Ziffern')
plt.title('Samples pro Ziffer')
plt.show()

In [ ]:
#
# Analyse der Verteilung der Werte in den Features als Tabelle
#
iris_non.describe()

# Outliers und Anomalien in den Daten

In [ ]:
#
# Analyse der Verteilung der Werte in den Features als Boxplot (outliers)
#
iris_non.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
plt.show()

In [ ]:
#
# Mathematische Berechnung
#

In [ ]:
values = iris_non.values[:,:-1].astype(np.float64)

In [ ]:
z = np.abs(stats.zscore(values))
print(z)

In [ ]:
#
# Filtern aller Werte mit z-score >= 3
#
iris_non_noo = iris_non[(z < 3).all(axis=1)]

In [ ]:
iris_non_noo.describe()

In [ ]:
#
# Analyse der Verteilung der Werte in den Features als Boxplot (outliers)
#
iris_non_noo.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
plt.show()

In [ ]:
#
# Durch Clustering (Teil des KI Profi Kurses)
#
outlier_detection = DBSCAN( min_samples = 2, eps = 0.6 )
clusters = outlier_detection.fit_predict( iris_non_noo.values[:,:-1] )
list(clusters).count(-1)

In [ ]:
#
# Zu welchen clustern wurden die Samples zugeordnet? Outliers werden mit -1 markiert.
#
print(clusters)

# Korrelationen und Beziehungen zwischen den Features

In [ ]:
#
# Analyse der Verteilung der Werte in den Features als Histogram
#
iris_non_noo.hist()
plt.show()

In [ ]:
#
# Mathematische Analyse der Beziehungen zwischen den Features (Korrelation)
#
iris_non.corr()

In [ ]:
sns.heatmap(iris_non_noo.corr(),annot=True,cmap='Blues_r')

In [ ]:
#
# Darstellung der Beziehungen zwischen den Features als Matrix Plot
#
scatter_matrix(iris_non_noo)
plt.show()

In [ ]:
#
# Darstellung der Beziehungen zwischen den Features als Matrix Plot mit Unterscheidung der Klassen
# Eine wichtige Fragestellung ist dabei die Separierbarkeit
#
sns.pairplot(iris_non_noo,hue='class')

In [ ]:
#
# Neue Form der Darstellung der Zusammenhänge zwischen Features und Klassen
#

In [ ]:
iris_data = iris_non_noo.to_dict('records')
iris_data[:2]

In [ ]:
hip.Experiment.from_iterable(iris_data).display(force_full_width=True)

# Prüfung der Modellqualität und implizit auch der Trainingsdatenqualität

[https://scikit-learn.org/stable/modules/cross_validation.html](https://scikit-learn.org/stable/modules/cross_validation.html)

In [ ]:
# 
# Aufteilen in Training Daten und Testdaten
#
array = iris_non_noo.values
X = array[:,0:4]
Y = array[:,4]
validation_size = 0.01
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=42)

In [ ]:
#
# Test einer Reihe von Modellen gleichzeitig. Dabei werden mehrere Verteilungen von Trainingsdaten
# erzeugt und trainiert. Dadurch wird sichtbar, wenn die Verteilung der Trainingsdaten nicht ausreichend
# breit ist (hohe Varianz). Mehr zu weiteren Methoden unter:
# https://www.pluralsight.com/guides/validating-machine-learning-models-scikit-learn
#

scoring = 'accuracy'

# Modelle anlegen
models = []
models.append(('KNN', KNeighborsClassifier(n_neighbors=5,metric='euclidean')))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

In [ ]:
#
# Testen mit mehreren unterschiedlichen zufälligen Aufteilungen der Daten
#
results = []
names = []
for name, model in models:
    
    kfold = model_selection.KFold( n_splits=10, random_state=42,shuffle=True)
    
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    print("Modell {}: accuracy {:.3f} (deviation {:.3f})".format(name, cv_results.mean(), cv_results.std()))

In [ ]:
#
# Testen mit mehreren unterschiedlichen Aufteilungen der Daten wobei die Klassenverteilung gleich bleibt
#
results = []
names = []
for name, model in models:
    
    skfold = model_selection.StratifiedKFold(n_splits=10, random_state=42,shuffle=True)
    
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=skfold, scoring=scoring)
    print("Modell {}: accuracy {:.3f} (deviation {:.3f})".format(name, cv_results.mean(), cv_results.std()))


# Untersuchung der Trainingsdaten


In [ ]:
#
# Laden der MNIST Daten und des Modelles
#
from keras.datasets import mnist
from keras.utils import to_categorical


# MNIST Daten mit Transformationen
(x_train, y_train), (_, _) = mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_train = x_train.astype('float32')
x_train = x_train / 255.0
y_train = to_categorical(y_train, num_classes=10)

In [ ]:
#
# Anzeige der Anzahl und Form der Samples
#
print('Trainingsdaten: X=%s, y=%s' % (x_train.shape, y_train.shape))

In [ ]:
# Modell aus Beispiel 4
from keras.models import model_from_json
prefix = 'results/04_'
modelName = prefix + "model.json"
weightName = prefix + "model.h5"
json_file = open(modelName, 'r')
loaded_model_json = json_file.read()
json_file.close()
# model aus json
loaded_model = model_from_json(loaded_model_json)
# gewichte aus h5 file
loaded_model.load_weights(weightName)
print("loaded model from disk")

In [ ]:
image = x_train[0].reshape((1,28,28,1))    
prediction_activation = loaded_model.predict([image])
predictedClass = np.argmax ( prediction_activation[0] )
confidence = prediction_activation[0][predictedClass]
predictedClass

In [ ]:
#
# Untersuchung aller Trainingsdaten mit Modell und Suche nach niedriger confidence
#

predictionConfidencePerClass = [[] for i in range(10)]
errorCount = 0
errorCountDistribution = [0] * 10
suspectList = []
suspectListConfidence = []

for i in range( x_train.shape[0] ):
    
    correctClass = np.argmax(y_train[i])
    image = x_train[i].reshape((1,28,28,1))
    prediction_activation = loaded_model.predict([image])
    predictedClass = np.argmax ( prediction_activation[0] )
    confidence = prediction_activation[0][predictedClass]
    if predictedClass != correctClass:
        errorCountDistribution[correctClass] = errorCountDistribution[correctClass] + 1
    else:
        if confidence < 0.9:
            predictionConfidencePerClass[correctClass].append(confidence)
            errorCount+= 1
            
        if confidence < 0.6:
            suspectList.append(i)
            suspectListConfidence.append(confidence)
            

In [ ]:
print('Anzahl der gefundenen Fälle ist {}'.format(errorCount))

In [ ]:
predictionConfidencePerClassNP = np.asarray(predictionConfidencePerClass)

In [ ]:
for clazzz in range(10):
    # Subset to the airline
    
    subset = predictionConfidencePerClassNP[clazzz]
    
    # Draw the density plot
    sns.distplot(subset, hist = False, kde = True, kde_kws = {'linewidth': 3}, label = 'Ziffer {}'.format(clazzz) )
    
# Plot formatting
plt.legend(prop={'size': 10}, title = 'digits')
plt.title('Confidence of classifications below 0.9')
plt.xlabel('confidence')
plt.ylabel('density')
plt.show()

In [ ]:
fig = plt.figure()

fig.suptitle('Suspect cases')
plotCount = 0

for i in range(len(suspectList)):
    correctClass = np.argmax(y_train[i])
    if plotCount < 9 and correctClass == 3:
            ax = plt.subplot(330 + 1 + plotCount)
            ax.set_title('susp {} conf {:.2f} digit {}'.format ( i, suspectListConfidence[i], correctClass ) )     
            image = x_train[i].reshape((28,28))
            plt.imshow(image, cmap=plt.get_cmap('gray'))    
            plotCount+= 1
        

plt.subplots_adjust(wspace=0.4,hspace=0.4)
plt.show()